In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pylab as plt 
import re
%matplotlib inline
data_path = "C:/Users/SpiffyApple/Documents/USC/Courses/Fall2016/PPD599/project/DATA_Nov4"

In [2]:
#data_files = ['acs'+str(x)+'.csv' for x in np.linspace(2006, 2015, 11).astype(int)]
data_files = os.listdir(data_path)
data_dict = {}

In [3]:
data_files

['1980.csv',
 '1980.xlsx',
 '1990.csv',
 '1990.xlsx',
 '2000.csv',
 '2000.xlsx',
 '2006.csv',
 '2006.xlsx',
 '2007.csv',
 '2007.xlsx',
 '2008.csv',
 '2008.xlsx',
 '2009.csv',
 '2009.xlsx',
 '2010.csv',
 '2010.xlsx',
 '2011.csv',
 '2011.xlsx',
 '2012.csv',
 '2012.xlsx',
 '2013.csv',
 '2013.xlsx',
 '2014.csv',
 '2014.xlsx',
 '2015.csv',
 '2015.xlsx',
 'acs_2006_2015.csv',
 'ca_data_1990_2015.csv',
 'Population Estimates 1990-2000.csv',
 'Population Estimates 2000-2010.csv',
 'pop_haus_1990-2010.csv']

In [4]:
for data_file in [x for x in data_files if "Population Estimates" in x ]:
    #print(data_file)
    key = re.sub("-| ", "_", data_file).lower()
    key = key.strip(".csv")
    print(key)
    """
    if "Population Estimates" in data_file:
        #data_dict[key] = pd.read_csv("/".join([data_path, data_file]), header=0).dropna(how='all').replace("%|,", "", regex=True)
        #data_dict[key].iloc[:,0] = data_dict[key].iloc[:,0].str.strip()
        #data_dict[key].set_index(data_dict[key].columns[0],inplace=True)
        #data_dict[key].columns = data_dict[key].columns.str.extract("\d+/\d+/(\d{4})", expand=False)
        #data_dict[key].index.name = "city"
    """    
    if "Population Estimates" in data_file:
        data_dict[key] = pd.read_csv("/".join([data_path, data_file]), header=0).dropna(how='all').replace("%|,", "", regex=True)
        data_dict[key].columns = data_dict[key].columns.str.lower()
        data_dict[key].county.fillna(inplace=True, method='ffill', axis=0)
        data_dict[key].date = data_dict[key].date.str.extract("\d+/\d+/(\d{4})", expand=False)
        #drop_above =data_dict[key][data_dict[key][data_dict[key].columns[0]] == 'Alameda'].index[0]
        #data_dict[key] = data_dict[key].iloc[drop_above:, :]

        data_dict[key].loc[data_dict[key].date == re.findall("(\d{4})_\d+",key)[0], 'county'] = np.nan
        data_dict[key].county.fillna(method = 'bfill', inplace=True)
        data_dict[key].set_index(data_dict[key].columns[0], inplace=True)
        data_dict[key].index.name = 'city'        

 

population_estimates_1990_2000
population_estimates_2000_2010


In [5]:
data_dict['population_estimates_2000_2010'].drop(["Balance of State", 'Incorporated', 'California', 'State Total', 'Calfornia', 'County Total'], inplace=True)
data_dict['population_estimates_1990_2000'].drop(["Balance of State", 'Incorporated', 'California', 'State Total', 'Calfornia', 'County Total'], inplace=True)

In [6]:
print(data_dict['population_estimates_2000_2010'].columns)
data_dict['population_estimates_1990_2000'].columns = [re.sub("-", "", x) for x in data_dict['population_estimates_1990_2000'].columns]
data_dict['population_estimates_1990_2000'].rename(columns={'person per household':'persons per household'}, inplace=True)
print(data_dict['population_estimates_1990_2000'].columns)


Index(['date', 'total', 'household', 'group quarters', 'total.1', 'single',
       'multiple', 'mobile homes', 'occupied', 'vacant units', 'vacancy rate',
       'persons per household'],
      dtype='object')
Index(['date', 'total', 'household', 'group quarters', 'total.1', 'single',
       'multiple', 'mobile homes', 'occupied', 'vacancy rate',
       'persons per household'],
      dtype='object')


In [7]:
df = pd.concat([data_dict['population_estimates_1990_2000'],  data_dict['population_estimates_2000_2010']])

In [8]:
df.reset_index(inplace=True)
df.drop_duplicates(['city', 'date'], keep='last', inplace=True)

In [9]:
#df.reset_index(inplace=True)
ml_index = pd.MultiIndex.from_arrays([df.city,df.date])
df.set_index(ml_index, inplace=True, drop=True)
df.sort_index(inplace=True)

In [10]:
df.drop(['date','city'], axis=1, inplace=True)

In [11]:
df.rename(columns = {'total':'population', 'total.1':'housing_units'}, inplace=True)

In [12]:
df.loc[:,'vacancy rate'] = df.loc[:,'vacancy rate'].astype(float)/100

In [13]:
df.index.names = ['city', 'year']

In [14]:
df.to_csv("/".join([data_path, 'pop_haus_1990-2010.csv']))

## Merge the datasets

In [35]:
acs = pd.read_csv("/".join([data_path, "acs_2006_2015.csv"]))
acs.year = acs.year.astype(str)

In [37]:
multi_idx = pd.MultiIndex.from_arrays([acs.year, acs.city])
acs.set_index(multi_idx, inplace=True)
acs.drop(['city', 'year'],axis=1 ,inplace=True)

In [38]:
acs.head()

foreign born  \
year city                        
2006 Alameda           15685.0   
     Alhambra          42893.0   
     Anaheim          131014.0   
     Antioch           20041.0   
     Bakersfield       53858.0   

                  native hawaiian and other pacific islander alone householder  \
year city                                                                        
2006 Alameda                                                    NaN              
     Alhambra                                                   NaN              
     Anaheim                                                42005.0              
     Antioch                                                57172.0              
     Bakersfield                                                NaN              

                  bachelor's degree  45 to 54 years  naturalized citizen  \
year city                                                                  
2006 Alameda                14332.0         11693.0              10575.0   
     Alhambra               14316.0         13186.0              24948.0   
     Anaheim                35495.0         40218.0              51541.0   
     Antioch                 9579.0         16238.0              10088.0   
     Bakersfield            24509.0         35824.0              21808.0   

                  5 to 9  hh45to50  hh100to125  not computed  800 to 999  \
year city                                                                  
2006 Alameda      2360.0     832.0      3420.0         675.0         NaN   
     Alhambra     5028.0    1364.0      2668.0         642.0      5259.0   
     Anaheim      8083.0    4362.0      9685.0        1395.0     10153.0   
     Antioch       844.0     871.0      4218.0          64.0         NaN   
     Bakersfield  6301.0    4207.0      7885.0        1727.0      7830.0   

                   ...     boat, rv, van, etc.  prop_black  high_or_less  \
year city          ...                                                     
2006 Alameda       ...                   201.0         NaN      0.256232   
     Alhambra      ...                     0.0    0.014493      0.421430   
     Anaheim       ...                    51.0    0.026602      0.499447   
     Antioch       ...                     0.0    0.158304      0.406578   
     Bakersfield   ...                    46.0    0.074806      0.475226   

                  rent_rate  rent_to_income  unemployment_rate  foreign_rate  \
year city                                                                      
2006 Alameda       0.477781        0.016136           0.085906      0.223408   
     Alhambra      0.560175        0.019853           0.079159      0.525051   
     Anaheim       0.458505        0.020603           0.058422      0.380699   
     Antioch       0.227266        0.015624           0.074095      0.203814   
     Bakersfield   0.376448        0.015850           0.077400      0.175928   

                    ami80    ami60    ami40  
year city                                    
2006 Alameda      54040.8  40530.6  27020.4  
     Alhambra     39852.8  29889.6  19926.4  
     Anaheim      44576.0  33432.0  22288.0  
     Antioch      53404.0  40053.0  26702.0  
     Bakersfield  41136.8  30852.6  20568.4  

[5 rows x 112 columns]

In [39]:
acs.index.levels[0]

Index(['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015'],
      dtype='object', name='year')

In [52]:
ca_df = df.reset_index().merge(acs.reset_index(), on=['city', 'year'], how='outer')

In [53]:
#ca_df = pd.concat([df, acs], axis=1)

In [54]:
ca_df.shape

(10699, 125)

In [55]:
ca_df.head()

,city,year,group quarters,household,mobile homes,multiple,occupied,persons per household,single,population,...,"boat, rv, van, etc.",prop_black,high_or_less,rent_rate,rent_to_income,unemployment_rate,foreign_rate,ami80,ami60,ami40
0,Adelanto,1990,0,6791,381,1306,2459,2.762,1066,6791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adelanto,1991,2,7121,384,1294,2471,2.882,1110,7123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adelanto,1992,425,8008,413,1278,2696,2.970,1375,8433,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adelanto,1993,397,10279,424,1262,3400,3.023,2195,10676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelanto,1994,435,12273,463,1246,3881,3.162,2738,12708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
ca_df.to_csv("/".join([data_path, "ca_data_1990_2015.csv"]))

### merging datasets with census

In [66]:
census_df = pd.read_csv("/".join([data_path, 'census_data_1980_2015.csv']))

In [67]:
census_df.year = census_df.year.astype(str)

In [76]:
ca2_df = df.reset_index().merge(census_df, on=['city', 'year'], how='outer')
ca2_df.shape

(10988, 131)

In [78]:
ca2_df.reset_index().drop_duplicates(['city', 'year']).to_csv("/".join([data_path, 'ca_data_1980_2015.csv']))

In [70]:
ca2_df.to_csv("/".join([data_path, 'ca_data_1980_2015.csv']))

In [75]:
ca2_df.head()

,city,year,group quarters,household,mobile homes,multiple,occupied_x,persons per household,single,population,...,two races,two races householder,under 5 years,unemployed,unemployment_rate,vacant,white alone,white alone householder,"white alone householder, not hispanic or latino",widowed
0,Adelanto,1990,0,6791,381,1306,2459,2.762,1066,6791,...,NaN,NaN,NaN,510.0,0.183058,346.0,5430.0,NaN,NaN,199.0
1,Adelanto,1991,2,7121,384,1294,2471,2.882,1110,7123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adelanto,1992,425,8008,413,1278,2696,2.970,1375,8433,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adelanto,1993,397,10279,424,1262,3400,3.023,2195,10676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelanto,1994,435,12273,463,1246,3881,3.162,2738,12708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
